In [1]:
import os
from crewai import Agent
from crewai import Task
from crewai import Crew, Process
from crewai_tools import BaseTool
from duckduckgo_search import DDGS

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"

In [3]:
os.environ["OPENAI_MODEL_NAME"]

'gpt-3.5-turbo'

In [4]:
class NewsSearchTool(BaseTool):
    name: str = "Latest News Search tool"
    description: str = "News search tool that searches the latest news on the given topic from internet and provides the results."

    def _run(self, topic: str) -> str:
        ddgs = DDGS()
        results = ddgs.news(keywords=topic, timelimit="d", max_results=20)
        # Your tool's logic here
        result_string=""
        for result in results:
            result_string += f"Title: {result['title']} \n Description: {result['body']}\n\n"
        if len(result_string.strip()) < 2:
            result_string="No news found for the given keyword"
        return result_string.strip()

news_search_tool = NewsSearchTool()

In [5]:
# Creating a researcher agent with memory and verbose mode
researcher = Agent(
  role='Researcher',
  goal='Get the latest news in {topic}',
  verbose=True,
  memory=True,
  backstory=(
      "As a knowledge enthusiast, you always look for new happenings on a given topic "
      "and share the knowledge with broader community."
  ),
  tools=[news_search_tool],
)

In [6]:
# Creating a writer agent with custom tools and delegation capability
summarizer = Agent(
  role='Summarizer',
  goal='Summarize the latest news in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you "
    "remove redundant information and summarize the "
    "discoveries in an accessible manner."
  ),
  allow_delegation=False
)

In [7]:
# Research task
research_task = Task(
  description=(
    "Get the latest news in {topic}."
  ),
  expected_output='Points covering the latest news on {topic}.',
  tools=[news_search_tool],
  agent=researcher,
)

# Summarizer task 
summarize_task = Task(
  description=(
      "Given all the news about the {topic}, summarize them to 2 or 3 bullet points. Discard the news which do not have central focus on {topic}."
      "If no news is relevant, return that no news found for the {topic}"
  ),
  expected_output='2 or 3 summarized bullet points, each one focussing on one specific news about the {topic}. If no news found, return no news found',
  agent=summarizer,
  output_file='news_summary.md',  # Example of output customization
)

In [8]:
# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, summarizer],
  tasks=[research_task, summarize_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
)

In [9]:
result = crew.kickoff(inputs={'topic': 'HDFC Bank'})
print(result)



> Entering new CrewAgentExecutor chain...
I need to use the "Latest News Search tool" to get the latest news on HDFC Bank.

Action:
Latest News Search tool

Action Input:
{
  "topic": "HDFC Bank"
}
 

Title: HDFC Bank poised for bullish breakout, says CLSA analyst 
 Description: CLSA's Laurence Balanco predicts bullish breakout for HDFC Bank stock, targeting 35-38% increase if resistance levels are surpassed.

Title: Regulatory concerns, bank charges force many fintechs to pause rent payments via credit cards 
 Description: There has been no official communication from RBI to any of the ecosystem partners however, the regulator's concerns stem from a product being used for unintended purposes

Title: Nifty, Nifty Bank, CPSE rejigs today; all eyes on NTPC, ITC, HDFC Bank, BEL, RIL shares 
 Description: Bharti Airtel, NTPC Ltd, Kotak Mahindra Bank Ltd, and ITC Ltd could attract net passive inflows to the tune of $36-$52 million due to adjustments while others such as Bharat Electronics

In [10]:
print(result)

1. HDFC Bank stock poised for bullish breakout, with CLSA analyst predicting a 35-38% increase if resistance levels are surpassed.
2. HDFC Bank updates rewards on Tata Neu Infinity card for UPI transactions, implementing changes from August 1.
3. HDFC Bank among the laggards in stock market performance, along with Mahindra & Mahindra, Tata Steel, and JSW Steel.
